# Dataset

## 116 imagens de pacientes com e sem osteoporose

![title](Capture.PNG)

# 1. Predição baseada em features

## 1.1 Pré-processamento

In [ ]:
def descrever(nomeDoCSV, imClass, data):
    csv_file = open(nomeDoCSV, 'wb')
    writer = csv.writer(csv_file)
    i = 0
    writer.writerow(["Mean", "Variance", "Skewness", "Kurtosis"])

    # Se a imagem for colorida, converter para cinza
    for image in imClass:
        image = color.rgb2gray(image)
        # Estatística
        stats = describe(image, axis=None)
        # Resultado
        writer.writerow([stats.mean, stats.variance, stats.skewness, stats.kurtosis])
        data.append([stats.mean, stats.variance, stats.skewness, stats.kurtosis])
        i = i + 1

## 1.2 Processamento

In [ ]:
def avaliar(avaliador, hDataTrain, hTargetTrain, hDataEval, hTargetEval):
    clf = MLPClassifier(solver=avaliador, alpha=1e-5, random_state=1)

    clf.fit(hDataTrain, hTargetTrain)

    predito = clf.predict(hDataEval)
    print("Resultado: " + str(np.mean(predito == hTargetEval)))

    scores = cross_val_score(clf, hDataEval, hTargetEval, cv = 5, verbose = 0, scoring='accuracy')
    print("Validacao cruzada: " + str(np.mean(scores)))

## 1.3 Resultado

### 1.3.1: lbfgs - Validação cruzada: 0.5

### 1.3.2: sgd - Validação cruzada: 0.5

### 1.3.3: adam - Validação cruzada: 0.5

# 2. Redução de dimensionalidade com o PCA

## 2.1 Pré-processamento

In [ ]:
def pca(ar, numeroDeComponentes):
    pca = decomposition.PCA(n_components=numeroDeComponentes)
    pca.fit(ar)
    ar = pca.transform(ar)
    return ar

## 2.2 Processamento

In [ ]:
def avaliar(avaliador, multiplicador, imagensTreino, targetTreino, imagensTeste, targetTeste):
    numNeuronios = int(floor(sqrt(multiplicador * 2)))

    clf = MLPClassifier(hidden_layer_sizes = numNeuronios, solver=avaliador, alpha=1e-5, 
                        random_state=1)

    clf.fit(imagensTreino, targetTreino)

    predito = clf.predict(imagensTeste)
    print("# Resultado: " + str(np.mean(predito == targetTeste)))

    scores = cross_val_score(clf, imagensTeste, targetTeste, cv = 5, verbose = 0, scoring='accuracy')
    print("# Validacao cruzada: " + str(np.mean(scores)))
    return np.mean(scores), np.mean(predito == targetTeste)

## 2.3 Resultado

![title](grafico1.jpg)

![title](grafico2.jpg)

![title](grafico3.jpg)

# 3. Predição com dimensão total dos dados 

## 3.1 Pré-processamento

In [ ]:
def lerDados():
    imagensTreino = []
    imagensTeste = []
    targetTreino = []
    targetTeste = []

    lerImagens(0, imagensTreino, imagensTeste, targetTreino, targetTeste)
    lerImagens(1, imagensTreino, imagensTeste, targetTreino, targetTeste)

    imagensTreino = np.array(imagensTreino)
    imagensTeste = np.array(imagensTeste)
    targetTreino = np.array(targetTreino)
    targetTeste = np.array(targetTeste)

    imagensTreino = imagensTreino.reshape(len(imagensTreino), 400 * 400)
    imagensTeste = imagensTeste.reshape(len(imagensTeste), 400 * 400)

    return imagensTreino, imagensTeste, targetTeste, targetTreino

## 3.2 Processamento

In [ ]:
def avaliar(avaliador, imagensTreino, targetTreino, imagensTeste, targetTeste):
    print("# Solver: " + avaliador)
    clf = MLPClassifier(hidden_layer_sizes=200, solver=avaliador, alpha=1e-5, random_state=1)

    clf.fit(imagensTreino, targetTreino)

    predito = clf.predict(imagensTeste)
    print("# Resultado: " + str(np.mean(predito == targetTeste)))

    scores = cross_val_score(clf, imagensTeste, targetTeste, cv = 5, verbose = 0, scoring='accuracy')
    print("# Validacao cruzada: " + str(np.mean(scores)))
    return np.mean(scores), np.mean(predito == targetTeste)

## 3.3 Resultado

### Tanto para 100, 200 e 566 (raiz) neurônios na camada oculta, os resultados foram:

#### 3.3.1: lbfgs - Validação cruzada: 0.5

#### 3.3.2: sgd - Validação cruzada: 0.5

#### 3.3.3: adam - Validação cruzada: 0.5

# 4. Trabalhos futuros

## Esse artigo usa fingerprints originadas por transformadas de Fourier: http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=809173 

## Matrizes de co-ocorrencia

## Transformada de Haar: http://www.ucsp.edu.pe/sibgrapi2013/eproceedings/wtd/114645.pdf